In [42]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import FourierOpticsLib as FO
# %matplotlib inline

In [43]:
ds_path = "/scr/sci/mhayman/holodec/holodec-ml-data/"
ds_name = "synthetic_holograms_v02.nc"
ds = xr.open_dataset(ds_path+ds_name)
ds_base = ds_name.replace(".nc","")

In [1]:
hist_bin_edges = {"d":np.linspace(20,70,25)}

SyntaxError: unexpected EOF while parsing (<ipython-input-1-c6688d5aec32>, line 3)

In [ ]:
# define the functions to be applied to the fourier transformed data
ft_func = {'real':np.real,'imag':np.imag,'amplitude':np.abs}
ft_scale = {'real':255,'imag':255,'amplitude':255}  # rescaling factors
encoded_dtype = "int16"

In [45]:
# initialize the image Fourier Transform channels
image_ft = {}
for func in ft_func.keys():
    image_ft[func] = xr.DataArray(np.zeros(ds['image'].shape,dtype='float32'),
                        dims=['ysize','xsize','hologram_number'])

# initialize the particle size bins
#diam_bin_edges = np.linspace(20,70,25)  # define the particle size histogram bins
#diam_bin_centers = 0.5*(diam_bin_edges[:-1]+diam_bin_edges[1:])
#diam_histogram = xr.DataArray(np.zeros((ds['image'].shape[0],diam_bin_centers.size)),
#                              coords=[ds['hologram_number'],diam_bin_centers],
#                              dims=['hologram_number','diameter_bin_centers'])

hist_bin_centers = {}
histogram_shape = []
histogram_size = 1
hist_dims = ['hologram_number']
hist_coords = [ds['hologram_number']]
for param in hist_bin_edges.keys():
    hist_bin_centers[param] = 0.5*(diam_bin_edges[:-1]+diam_bin_edges[1:])
    histogram_shape+=[hist_bin_centers[param].size]
    histogram_size *= hist_bin_centers[param].size
    hist_dims += [param+'_bin_centers']
    
hist_histogram = xr.DataArray(np.zeros([ds.['image'].shape[0]]+histogram_shape),
                              coords=[ds['hologram_number']]+hist_bin_centers.values(),
                              dims=['hologram_number']+hist_dims)


In [32]:

ds_ft = ds_ft.assign(**hist_bin_centers) # need to change names to make this work appropriately
ds_ft = ds.assign(diam_histogram=diam_histogram)
ds_ft = ds_ft.assign_coords(diam_bin_edges=diam_bin_edges)

In [33]:
ds_ft

<xarray.Dataset>
Dimensions:               (diam_bin_edges: 25, diameter_bin_centers: 24, hologram_number: 10000, particle: 30000, xsize: 600, ysize: 400)
Coordinates:
  * hologram_number       (hologram_number) int64 0 1 2 3 ... 9997 9998 9999
  * diameter_bin_centers  (diameter_bin_centers) float64 21.04 23.12 ... 68.96
  * diam_bin_edges        (diam_bin_edges) float64 20.0 22.08 ... 67.92 70.0
Dimensions without coordinates: particle, xsize, ysize
Data variables:
    hid                   (particle) float32 1.0 1.0 1.0 ... 10000.0 10000.0
    d                     (particle) float32 ...
    x                     (particle) float32 ...
    y                     (particle) float32 ...
    z                     (particle) float32 ...
    image                 (hologram_number, xsize, ysize) uint8 ...
    image_ft              (hologram_number, xsize, ysize) float32 0.0 ... 0.0
    diam_histogram        (hologram_number, diameter_bin_centers) float64 0.0 ... 0.0
Attributes:
    DateCre

In [4]:
# store the Fourier Transform and particle size histogram for each hologram
for im in ds_ft['hologram_number'].values:
    particle_index = np.nonzero(ds_ft['hid'].values==im+1)  # find the particles in this hologram
    
    # make a histogram of particle sizes and store it in the data set
    hist0 = np.histogramdd(ds_ft)
    hist0 = np.histogram(ds_ft['d'].values[particle_index],bins=ds_ft['diam_bin_edges'])
    ds_ft['diam_histogram'][im,:] = hist0[0]
    
    # FT the image and store the desired operations
    image_ft0 = FO.OpticsFFT(ds['image'][:,:,im]-np.mean(ds['image'][:,:,im]))
    for func in ft_func.keys():
        image_ft[func][:,:,im] = ft_func[func](image_ft0) / ft_scale[func]

In [ ]:
image_ft = xr.concat(list(image_ft.values()),pd.Index(ft_func.keys(),name='channels'))
ds_ft = ds.assign(image_ft=image_ft)

In [41]:
encode_defs = {'image_ft':{'dtype':encoded_dtype,
                           'scale_factor':(image_ft.max()-image_ft.min()).values/(np.iinfo(encoded_dtype).max-np.iinfo(encoded_dtype).min),
                           'add_offset':image_ft.mean().values}}

(array([0, 1, 2]),)

In [6]:
ds_ft

<xarray.Dataset>
Dimensions:   (hologram_number: 10000, particle: 10000, xsize: 600, ysize: 400)
Dimensions without coordinates: hologram_number, particle, xsize, ysize
Data variables:
    hid       (particle) float32 ...
    d         (particle) float32 ...
    x         (particle) float32 ...
    y         (particle) float32 ...
    z         (particle) float32 ...
    image     (ysize, xsize, hologram_number) uint8 ...
    image_ft  (ysize, xsize, hologram_number) float32 0.71851695 ... 0.8084589
Attributes:
    DateCreated:           28-Oct-2019
    Nx:                    600.0
    Ny:                    400.0
    dx:                    2.96e-06
    dy:                    2.96e-06
    lambda:                3.55e-07
    Dpmin:                 2e-05
    Dpmax:                 7e-05
    zMin:                  0.014
    zMax:                  0.158
    NParticles:            1.0
    Nbits:                 8.0
    ccdelevel:             20000.0
    gainFactor:            24.0
    readN

In [7]:
ds_ft.to_netcdf(ds_path+ds_base+"_ft_ac_"+"_".join(ft_func.keys())+".nc",encoding=encode_defs)

In [ ]:
ds.close()
ds_ft.close()